In [1]:
# We'll first need the data from my drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Process

1. Create Intents
2. 

# Packages

In [2]:
path='/content/drive/My Drive/1000ml/Project 7 - Chatbot/Data/'

# We'll need the following nltk packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# We'll use lemmas instead of stems.
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# We will tag the parts of the speech 
from nltk.tag import pos_tag

# Get the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

# We'll need json to open a file of intents
import json

# We will need to save the model
import pickle

# Some helper libraries
import numpy as np
import pandas as pd
import random
import re

# We'll need these to actually create a machine learning model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adagrad, Adam, Adadelta

# We may need to try out a grid search
from sklearn.model_selection import GridSearchCV

# We'll need this for the in-code tagging of words
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import Matcher

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
m_tool = Matcher(nlp.vocab)

In [10]:
p1 = [{'LOWER': 'from'}, {'ENT_TYPE':'GPE'}]
p2 = [{'LOWER': 'to'}, {'ENT_TYPE': 'GPE'}]

In [11]:
m_tool.add('OrDest', None, p1, p2)

In [12]:
sentence = nlp(u'I want to book a flight from Toronto to Vancouver')

In [13]:
phrase_matches = m_tool(sentence)
print(phrase_matches )

[(14941694479039199002, 6, 8), (14941694479039199002, 8, 10)]


# Copy Paste from Article

In [3]:
# first we'll need the pairs of conversation:
data_file1 = open(f'{path}frames.json').read()
travel_txt = json.loads(data_file1)

In [4]:
user_txt=[]
robo_txt=[]
for convo in travel_txt:
  if len(convo['turns'])%2==0:
    for i in range(0, len(convo['turns']), 2):
      user_txt.append(convo['turns'][i]['text'])
      robo_txt.append(convo['turns'][i+1]['text'])
  else:
    for i in range(0, len(convo['turns'])-1, 2):
      user_txt.append(convo['turns'][i]['text'])
      robo_txt.append(convo['turns'][i+1]['text'])

In [5]:
# Cleaning Data
lines = [re.sub(r"\[\w+\]",'hi',line) for line in user_txt]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in robo_txt]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]

# grouping lines by response pair
pairs = list(zip(lines,lines2))
random.shuffle(pairs)

In [6]:
def create_tokens(pairs, start, end):
  input_docs = []
  target_docs = []
  input_tokens = set()
  target_tokens = set()

  for line in pairs[start:end]:
    input_doc, target_doc = line[0], line[1]
    # Appending each input sentence to input_docs
    input_docs.append(input_doc)
    # Splitting words from punctuation  
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
    # Redefine target_doc below and append it to target_docs
    target_doc = '<START> ' + target_doc + ' <END>'
    target_docs.append(target_doc)
    
    # Now we split up each sentence into words and add each unique word to our vocabulary set
    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
      if token not in input_tokens:
        input_tokens.add(token)
    for token in target_doc.split():
      if token not in target_tokens:
        target_tokens.add(token)

  input_tokens = sorted(list(input_tokens))
  target_tokens = sorted(list(target_tokens))
  num_encoder_tokens = len(input_tokens)
  num_decoder_tokens = len(target_tokens)
  return (input_docs, input_tokens, target_docs, target_tokens, num_encoder_tokens, num_decoder_tokens)

In [7]:
# The reason for this is we are running out of RAM. See if I can't chunk up the info, and combine it at the end.
input_docs1, input_tokens1, target_docs1, target_tokens1, num_encoder_tokens1, num_decoder_tokens1 = create_tokens(pairs,0,2500)
# input_docs2, input_tokens2, target_docs2, target_tokens2, num_encoder_tokens2, num_decoder_tokens2 = create_tokens(pairs,2500,5000)
# input_docs3, input_tokens3, target_docs3, target_tokens3, num_encoder_tokens3, num_decoder_tokens3 = create_tokens(pairs,5000,7500)
# input_docs4, input_tokens4, target_docs4, target_tokens4, num_encoder_tokens4, num_decoder_tokens4 = create_tokens(pairs,7500,len(pairs))

In [8]:
# Below we'll need two versions of the same dictionary for different purposes. Just with the keys and positions reversed.
def feature_dicts(input_tokens, target_tokens):
  input_features_dict = dict([(token, i) for i, token in enumerate(input_tokens)])

  target_features_dict = dict([(token, i) for i, token in enumerate(target_tokens)])

  reverse_input_features_dict = dict((i, token) for token, i in input_features_dict.items())

  reverse_target_features_dict = dict((i, token) for token, i in target_features_dict.items())

  return (input_features_dict, target_features_dict, reverse_input_features_dict, reverse_target_features_dict)

In [9]:
input_features_dict1, target_features_dict1, reverse_input_features_dict1, reverse_target_features_dict1 = feature_dicts(input_tokens1, target_tokens1)
# input_features_dict2, target_features_dict2, reverse_input_features_dict2, reverse_target_features_dict2 = feature_dicts(input_tokens2, target_tokens2)
# input_features_dict3, target_features_dict3, reverse_input_features_dict3, reverse_target_features_dict3 = feature_dicts(input_tokens3, target_tokens3)
# input_features_dict4, target_features_dict4, reverse_input_features_dict4, reverse_target_features_dict4 = feature_dicts(input_tokens4, target_tokens4)

In [10]:
def create_bow_data(input_docs, target_docs, input_features_dict, target_features_dict, num_encoder_tokens, num_decoder_tokens):
  #Maximum length of sentences in input and target documents
  max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
  max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

  # This is basically making a 3 dimensional bag of words
  encoder_input_data = np.zeros((len(input_docs), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
  decoder_input_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
  decoder_target_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

  for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
      for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
          #Assign 1. for the current line, timestep, & word in encoder_input_data
          encoder_input_data[line, timestep, input_features_dict[token]] = 1.
      # the second bit here is to incorporate "teaching forcing". A method of Recursive learning which helps a model learn, by comparing with the answer at each step.
      for timestep, token in enumerate(target_doc.split()):
          decoder_input_data[line, timestep, target_features_dict[token]] = 1.
          if timestep > 0:
              decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.
  return (encoder_input_data, decoder_input_data, decoder_target_data)

In [11]:
def save_large_arrays(input_docs, target_docs, input_features_dict, target_features_dict, num_encoder_tokens, num_decoder_tokens):
  with open('encoder_input1.npy', 'wb') as f:
    np.save(f, create_bow_data(input_docs1, target_docs1, input_features_dict1, target_features_dict1, num_encoder_tokens1, num_decoder_tokens1)[0], allow_pickle=True)
  with open('decoder_input1.npy', 'wb') as f:
    np.save(f, create_bow_data(input_docs1, target_docs1, input_features_dict1, target_features_dict1, num_encoder_tokens1, num_decoder_tokens1)[1], allow_pickle=True)
  with open('decoder_target1.npy', 'wb') as f:
    np.save(f, create_bow_data(input_docs1, target_docs1, input_features_dict1, target_features_dict1, num_encoder_tokens1, num_decoder_tokens1)[2], allow_pickle=True)  

In [12]:
encoder_input1, decoder_input1, decoder_target1 = create_bow_data(input_docs1, target_docs1, input_features_dict1, target_features_dict1, num_encoder_tokens1, num_decoder_tokens1)

In [13]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 100
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens1))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens1))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling 
training_model.compile(optimizer='Adagrad', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input1, decoder_input1], decoder_target1, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Train on 2000 samples, validate on 500 samples
Epoch 1/100
2000/2000 [==============================] - 506s 253ms/step - loss: 0.2914 - accuracy: 0.0797 - val_loss: 0.5452 - val_accuracy: 0.0404
Epoch 2/100
2000/2000 [==============================] - 509s 254ms/step - loss: 0.2269 - accuracy: 0.0934 - val_loss: 0.5491 - val_accuracy: 0.0405
Epoch 3/100
2000/2000 [==============================] - 549s 274ms/step - loss: 0.2036 - accuracy: 0.0983 - val_loss: 0.5550 - val_accuracy: 0.0407
Epoch 4/100
2000/2000 [==============================] - 575s 288ms/step - loss: 0.1885 - accuracy: 0.1012 - val_loss: 0.5612 - val_accuracy: 0.0395
Epoch 5/100
2000/2000 [==============================] - 556s 278ms/step - loss: 0.1769 - accuracy: 0.1027 - val_loss: 0.5646 - val_accuracy: 0.0396
Epoch 6/100
2000/2000 [==============================] - 559s 279ms/step - loss: 0.1673 - accuracy: 0.1047 - val_loss: 0.5683 - val_accuracy: 0.0393
Epoch 7/100
1400/2000 [====================>.........] - ET

KeyboardInterrupt: ignored